In [ ]:
!pip3 install torch
!pip3 install torchvision

import numpy as np
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Augograd
- torch.autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분 제공
- 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의 된다는 뜻
- backprop를 위해 미분값을 자동으로 계산

requires_grad 속성을 True로 설정하면, 해당 텐서에서 이뤄지는 모든 연산들을 추적하기 시작
기록을 추적하는 것을 중단하려면 .detach()를 호출하여 연산기록으로부터 분리

In [ ]:
a = torch.rand(3,3)
a.requires_grad_(True)

b = (a * a).sum()
print(b)
print(b.grad_fn)


tensor(6.4949, grad_fn=<SumBackward0>)


# Gradient

In [ ]:
x = torch.ones(3, 3, requires_grad=True)
print(x)

y = x + 5
print(y)

z = y*y
out = z.mean()
print(z, out)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]], grad_fn=<AddBackward0>)
tensor([[36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.]], grad_fn=<MulBackward0>) tensor(36., grad_fn=<MeanBackward0>)


계산이 완료 된후, backward()를 호출 하면 자동으로 역전파 계산이 가능하고 .grad속성에 누적됨

In [ ]:
print(out)
out.backward()

tensor(36., grad_fn=<MeanBackward0>)


grad : data 가 거쳐온 layer에 대한 미분값 저장

In [ ]:
print(x)
print(x.grad)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])


Example

In [36]:
a = torch.tensor([2., 3.], requires_grad= True)
b = torch.tensor([6., 4.], requires_grad= True)

Q = 3*a**3 - b**2
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

print(a.grad)
print(b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])


In [42]:
a = torch.tensor([10., 10., 10.], requires_grad= True)
y = a**2

print(y)

v = torch.tensor([0.1, 1.0, 0.0001])
y.backward(v)

print(a.grad)

tensor([100., 100., 100.], grad_fn=<PowBackward0>)
tensor([2.0000e+00, 2.0000e+01, 2.0000e-03])


with torch.no_grad()를 사용하여 기울기의 없데이트를 하지 않음
- 기록을 추적하는 것을 방지 하기 위해 with torch.no_grad() 블록으로 감싸면 기울기 계산을 필요 없지만, requires_grad=True로 설정되어 학습 가능한 매개변수를 갖는 모델 평가할떄 유용

# 자종 미분 흐름 예제
- a -> b -> c -> out
backward()를 통해 a<- b <- c<- out을 계산하면 out/a 값이 a.grad 가 채워짐

In [57]:
a = torch.ones(2,2, requires_grad=True)

b = a + 2
print(b)

c = b**2
print(c)

out = c.sum()
print(out)

out.backward()

print("============ a ==============")
print(a.data)
print(a.grad)
print(a.grad_fn)

print("============ b ==============")
print(b.data)
print(b.grad)
print(b.grad_fn)

print("============ c ==============")
print(c.data)
print(c.grad)
print(c.grad_fn)


print("============ out ==============")
print(out.data)
print(out.grad)
print(out.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[9., 9.],
        [9., 9.]], grad_fn=<PowBackward0>)
tensor(36., grad_fn=<SumBackward0>)
============ a ==============
tensor([[1., 1.],
        [1., 1.]])
tensor([[6., 6.],
        [6., 6.]])
None
============ b ==============
tensor([[3., 3.],
        [3., 3.]])
None
============ c ==============
tensor([[9., 9.],
        [9., 9.]])
None
============ out ==============
tensor(36.)
None
